In [26]:
# 모듈 import
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets.builder import DATASETS
from mmdet.datasets import CocoDataset

In [27]:
# 데이터셋 분할을 위한 커스텀 데이터셋 클래스 정의
@DATASETS.register_module()
class SplitCocoDataset(CocoDataset):
    def __init__(self, ann_file, pipeline, split='train', **kwargs):
        super().__init__(ann_file, pipeline, **kwargs)
        
        # 전체 이미지 ID 목록 가져오기
        self.img_ids = self.coco.getImgIds()
        
        # 8:2로 분할
        num_imgs = len(self.img_ids)
        num_train = int(num_imgs * 0.8)
        
        if split == 'train':
            self.img_ids = self.img_ids[:num_train]
        else:  # validation
            self.img_ids = self.img_ids[num_train:]

KeyError: 'SplitCocoDataset is already registered in dataset'

In [35]:
# custom config file 불러오기
cfg = Config.fromfile('/data/ephemeral/home/Jihwan/code/baseline/mmdetection/custom_faster_rcnn_swin-t-p4-w7-fpn_1x_coco.py')


# 데이터셋 경로 설정 (만약 config 파일에 이미 정의되어 있다면 이 부분도 생략 가능)
root='/data/ephemeral/home/Jihwan/code/dataset/'
cfg.data.train.img_prefix = root
cfg.data.train.ann_file = root + 'train.json'
cfg.data.val.img_prefix = root
cfg.data.val.ann_file = root + 'train.json'  # train.json을 사용
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json'

# 데이터셋 타입을 SplitCocoDataset으로 변경
cfg.data.train.type = 'SplitCocoDataset'
cfg.data.train.split = 'train'
cfg.data.val.type = 'SplitCocoDataset'
cfg.data.val.split = 'val'

# work_dir 설정 (선택적)
cfg.work_dir = './work_dirs/custom_faster_rcnn_swin_fpn_trash'


Exception ignored in: <function _TemporaryFileCloser.__del__ at 0x7f770711f0a0>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/tempfile.py", line 450, in __del__
    self.close()
  File "/opt/conda/lib/python3.10/tempfile.py", line 446, in close
    unlink(self.name)
FileNotFoundError: [Errno 2] No such file or directory: '/tmp/tmptka5usvu/tmpnnz0pwwy.py'


In [36]:
cfg.data.train.pipeline

[{'type': 'LoadImageFromFile'},
 {'type': 'LoadAnnotations', 'with_bbox': True},
 {'type': 'Resize', 'img_scale': (1333, 800), 'keep_ratio': True},
 {'type': 'RandomFlip', 'flip_ratio': 0.5},
 {'type': 'Normalize',
  'mean': [123.675, 116.28, 103.53],
  'std': [58.395, 57.12, 57.375],
  'to_rgb': True},
 {'type': 'Pad', 'size_divisor': 32},
 {'type': 'DefaultFormatBundle'},
 {'type': 'Collect', 'keys': ['img', 'gt_bboxes', 'gt_labels']}]

In [37]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.08s)
creating index...
index created!


In [31]:
# dataset 확인
datasets[0]


SplitCocoDataset Train dataset with number of images 4883, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 3965  | 1 [Paper]     | 6352  | 2 [Paper pack]  | 897   | 3 [Metal]   | 936   | 4 [Glass]    | 982   |
| 5 [Plastic]       | 2943  | 6 [Styrofoam] | 1263  | 7 [Plastic bag] | 5178  | 8 [Battery] | 159   | 9 [Clothing] | 468   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [39]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

AttributeError: FasterRCNN: 'NoneType' object has no attribute 'rpn'

In [25]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=False)

NameError: name 'model' is not defined